In [2]:
%cd drive/MyDrive/data

/content/drive/MyDrive/data


In [4]:
import pandas as pd

data = pd.read_csv("test.csv")
data.head()

,ID,Utterance,Speaker,Dialogue_ID
0,TEST_0000,Why do all the coffee cups have figures below?,Mark,0
1,TEST_0001,"Oh. It's so Monica can follow. Of this way, if...",Rachell,0
2,TEST_0002,You know what?,Rachell,0
3,TEST_0003,"Come on, Lydia, you can do it.",Joeyy,1
4,TEST_0004,To push!,Joeyy,1


In [ ]:
speak = data['Speaker'].tolist()
speak

In [ ]:
text = data['Utterance'].tolist()
text

In [7]:
conv_text = []

for i in range(len(data)):
  conv_text.append(speak[i] + " : "+ text[i])

In [8]:
conv_text[:4]

['Mark : Why do all the coffee cups have figures below?',
 'Rachell : Oh. It\'s so Monica can follow. Of this way, if one on them is missing, she can say to herself: "Where is the number 27?"',
 'Rachell : You know what?',
 'Joeyy : Come on, Lydia, you can do it.']

In [35]:
train = []
for sent in conv_text:
    convo = sent.split(':')
    if len(convo) > 2:
        convo[1] += ('-').join(convo[2:])
    convo = convo[:2]
    convo[1] = convo[1].strip()
    if not convo[1] is '':
        train.append([c for c in convo])

In [36]:
train[:4]

[['Mark ', 'Why do all the coffee cups have figures below?'],
 ['Rachell ',
  'Oh. It\'s so Monica can follow. Of this way, if one on them is missing, she can say to herself "Where is the number 27?"'],
 ['Rachell ', 'You know what?'],
 ['Joeyy ', 'Come on, Lydia, you can do it.']]

In [ ]:
import json
with open("contract.json", "r") as read_file:
    data = json.load(read_file)
    
cont = data['appos']
cont

In [39]:
import re

process_train = []
for speaker, dialog in train:
    for (key, val) in cont.items():
        dialog = dialog.lower()
        dialog = dialog.replace(key, val)
        dialog = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", dialog)
        dialog = ' ' + dialog + ' '
        dialog = re.sub(' +', ' ',dialog)
        
    process_train.append([speaker, dialog])

In [40]:
process_train[:4]

[['Mark ', ' why do all the coffee cups have figures below ? '],
 ['Rachell ',
  ' oh . it\'s so monica can follow . of this way , if one on them is missing , she can say to herself " where is the number 27 ?" '],
 ['Rachell ', ' you know what ? '],
 ['Joeyy ', ' come on , lydia , you can do it . ']]

In [14]:
with open("thirdperson.json", "r") as read_file:
    data = json.load(read_file)

thirdperson = {}

for (key, value) in data['thirdperson'].items():
    thirdperson[' ' + key + ' '] = ' ' + value + ' '

In [32]:
input_data = []

previousSpeaker = ''
for speaker, dialog in process_train:
    for (key, val) in thirdperson.items():
        dialog = dialog.lower()
        dialog = dialog.replace(key, val)
        dialog = dialog.replace(' i ', ' ' +speaker + ' ')
        dialog = dialog.replace('i ', ' ' +speaker + ' ')
        dialog = dialog.replace('you', previousSpeaker if not previousSpeaker is '' else 'them' )
    input_data.append([speaker, dialog])

In [33]:
input_data[:4]

[['Mark ', ' why did all the coffee cups had figures below ? '],
 ['Rachell ',
  ' oh . it\'s so monica can follow . of this way , if one on them was missing , she can say to herself " where was the number 27 ?" '],
 ['Rachell ', ' them know what ? '],
 ['Joeyy ', ' come on , lydia , them can did it . ']]

In [17]:
!pip install transformers -q

     |████████████████████████████████| 5.5 MB 27.3 MB/s 
     |████████████████████████████████| 182 kB 82.9 MB/s 
     |████████████████████████████████| 7.6 MB 75.1 MB/s 


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer
from transformers import pipeline

model_name = "tae898/emoberta-large"

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/408 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [20]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [42]:
pred = classifier(conv_text)

In [44]:
pred_target = []

for i in range(len(pred)):
  pred_target.append(pred[i]['label'])

In [45]:
sample = pd.read_csv("sample_submission.csv")
sample['Target'] = pred_target
sample.to_csv("emoberta_submit_3.csv", index = False)